<a href="https://colab.research.google.com/github/Prathamesh0510/Face_reco/blob/main/PANOVR_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP PACKAGES & MODULES

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## WHISPER AI

In [ ]:
!pip install pydub

In [ ]:
import ipywidgets as widgets
from IPython import display as disp
from IPython.display import display, Audio, clear_output
from google.colab import output
import base64
from pydub import AudioSegment
import io
import tempfile
import librosa


import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
from google.colab import output
import base64
from pydub import AudioSegment
import io
import tempfile
import librosa

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q
import whisper

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
whisper_model = whisper.load_model("large")

Speech Record Function

In [ ]:
def record_audio(seconds=3,
                 sample_rate=44100,
                 normalize_db=0.1,
                 save_path=None):
    # Use Javascript to record audio.
    record_js_code = """
      const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
      const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onloadend = e => resolve(e.srcElement.result)
        reader.readAsDataURL(blob)
      })
      var record = time => new Promise(async resolve => {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true })
        recorder = new MediaRecorder(stream)
        chunks = []
        recorder.ondataavailable = e => chunks.push(e.data)
        recorder.start()
        await sleep(time)
        recorder.onstop = async ()=>{
          blob = new Blob(chunks)
          text = await b2text(blob)
          resolve(text)
        }
        recorder.stop()
      })
      """
    print('Starting recording for {} seconds...'.format(seconds))
    display(disp.Javascript(record_js_code))
    audio_string = output.eval_js('record(%d)' % (seconds * 1000.0))
    print('Finished recording!')
    audio_bytes = base64.b64decode(audio_string.split(',')[1])
    audio_np = audio_bytes_to_np(audio_bytes, sample_rate=sample_rate, normalize_db=normalize_db)

    if save_path:
        save_audio(audio_bytes, save_path)

    return audio_np

In [ ]:
def audio_bytes_to_np(wav_data, sample_rate=44100, normalize_db=0.1):
    # Parse and normalize the audio.
    audio = AudioSegment.from_file(io.BytesIO(wav_data))
    audio.remove_dc_offset()
    if normalize_db is not None:
        audio.normalize(headroom=normalize_db)
    # Save to tempfile and load with librosa.
    with tempfile.NamedTemporaryFile(suffix='.wav') as temp_wav_file:
        fname = temp_wav_file.name
        audio.export(fname, format='wav')
        audio_np, unused_sr = librosa.load(fname, sr=sample_rate)
    return audio_np

def save_audio(audio_bytes, save_path):
    with open(save_path, 'wb') as f:
        f.write(audio_bytes)

SAMPLE_RATE = 44100
record_seconds = 10  # Adjust record_seconds as needed

def _record_audio(b):
    global audio
    clear_output()
    audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE, save_path="recorded_audio.wav")
    display(Audio(audio, rate=SAMPLE_RATE))

## GPT NEO

### FINE TUNING (FIRST RUN)

Cleaning and parsing dataset

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00


In [ ]:
def correct_dataset(file_path):
    corrected_lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if ": " not in line:
            # Attempt to correct lines that do not contain ": " by finding the first colon.
            colon_index = line.find(":")
            if colon_index != -1:
                # Insert a space after the colon if it's missing.
                line = line[:colon_index + 1] + " " + line[colon_index + 1:]
            else:
                print(f"Line {i+1} does not contain a colon and may need manual correction: {line}")
                continue  # Skip lines that can't be automatically corrected
        corrected_lines.append(line)

    # Write the corrected lines to a new file
    with open(file_path.replace('.txt', '_corrected.txt'), 'w', encoding='utf-8') as file:
        file.writelines(corrected_lines)
    print(f"Corrected file saved as: {file_path.replace('.txt', '_corrected.txt')}")

# Call the function with the path to your file
file_path = '/content/drive/MyDrive/PANOVR/PromptDataset_v1/PROMPT DATASET.txt'  # Update with your actual file path
correct_dataset(file_path)

Corrected file saved as: /content/drive/MyDrive/PANOVR/PromptDataset_v1/PROMPT DATASET_corrected.txt


In [ ]:
def parse_dataset(file_path):
    categories = []
    descriptions = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line in lines:
        if ": " in line:  # Check if the line contains a colon and space
            category, description = line.split(": ", 1)
            categories.append(category.strip())
            descriptions.append(description.strip())
        else:
            print(f"Skipping line because it does not contain a colon followed by a space: {line}")
    return categories, descriptions

# Now call the parse_dataset function with your file path
file_path = '/content/drive/MyDrive/PANOVR/PromptDataset_v1/PROMPT DATASET_corrected.txt'
categories, descriptions = parse_dataset(file_path)

Save GPT NEO TO DRIVE

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model_name = "EleutherAI/gpt-neo-125M"  # Example model
model2 = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Save the model and tokenizer to your Drive
model_save_path = "/content/drive/My Drive/PANOVR/Models/GPT NEO"
model2.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

('/content/drive/My Drive/PANOVR/Models/GPT NEO/tokenizer_config.json',
 '/content/drive/My Drive/PANOVR/Models/GPT NEO/special_tokens_map.json',
 '/content/drive/My Drive/PANOVR/Models/GPT NEO/vocab.json',
 '/content/drive/My Drive/PANOVR/Models/GPT NEO/merges.txt',
 '/content/drive/My Drive/PANOVR/Models/GPT NEO/added_tokens.json')

Train NEO Model with Custom Dataset for prompt Generation

In [ ]:
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from transformers import GPT2Tokenizer
from torch.optim import AdamW


tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the descriptions
inputs = tokenizer(descriptions, max_length=40, padding='max_length', truncation=True, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Create the dataset
dataset = TensorDataset(input_ids, attention_mask)
train_dataloader = DataLoader(dataset, batch_size=10, shuffle=True)  # Adjust batch size if necessary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)

optimizer = AdamW(model2.parameters(), lr=5e-5)
epochs = 5  # Adjust based on your dataset size and desired training duration
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * epochs)

In [ ]:
# Assuming your model and optimizer are already set up and your DataLoader is named train_dataloader
model2.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        # Since the batch is a tuple with input_ids and attention_mask, we unpack it here
        input_ids, attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Forward pass
        outputs = model2(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss

        # Backward and optimize
        loss.backward()
        optimizer.step()
        scheduler.step()  # Update the learning rate schedule
        optimizer.zero_grad()

        # Add the loss to the total loss
        total_loss += loss.item()
    print(f"Epoch {epoch+1} | Loss: {total_loss / len(train_dataloader)}")

Epoch 1 | Loss: 1.77619204426756
Epoch 2 | Loss: 1.2074650201467003
Epoch 3 | Loss: 0.9865285164058799
Epoch 4 | Loss: 0.843404222242903
Epoch 5 | Loss: 0.7568034736236723


In [ ]:
import os

# Define the absolute path to where you want to save the model.
model_save_path_fine = '/content/drive/MyDrive/PANOVR/Models/GPT NEO FINE TUNED'

# Create the directory if it doesn't exist.
if not os.path.exists(model_save_path_fine):
    os.makedirs(model_save_path_fine)

# Save the model and tokenizer to the defined path.
model2.save_pretrained(model_save_path_fine)
tokenizer.save_pretrained(model_save_path_fine)

# Use the exact same path to list the contents.
!ls -lha "{model_save_path_fine}"

total 480M
-rw------- 1 root root 1.1K Apr  6 14:42 config.json
-rw------- 1 root root  119 Apr  6 14:42 generation_config.json
-rw------- 1 root root 446K Apr  6 14:42 merges.txt
-rw------- 1 root root 478M Apr  6 14:42 model.safetensors
-rw------- 1 root root  470 Apr  6 14:42 special_tokens_map.json
-rw------- 1 root root  525 Apr  6 14:42 tokenizer_config.json
-rw------- 1 root root 976K Apr  6 14:42 vocab.json


### FINE TUNED MODEL (SECOND RUN)

In [ ]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# Replace with your actual model path
model_path = '/content/drive/MyDrive/PANOVR/Models/GPT NEO FINE TUNED'

# Load the fine-tuned model and tokenizer
model3 = GPTNeoForCausalLM.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Make sure to set the tokenizer's padding token if you plan to use padded sequences
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def generate_text(prompt, model3, tokenizer, max_length=30):
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    input_ids = input_ids.to(next(model3.parameters()).device)  # Move input to the same device as model

    # Generate text
    output_sequences = model3.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=1.0,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1
    )

    # Decode the output
    generated_sequence = output_sequences[0].tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    text = text.replace(tokenizer.eos_token, '').strip()

    return text

# Test the function with a custom prompt
test_prompt = "The waterfall landscape"
print(generate_text(test_prompt, model3, tokenizer))

test_prompt = "From the top of a cliff"
print(generate_text(test_prompt, model3, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The waterfall landscape is reminiscent of a medieval castle, with turrets and towers adorned with decorative panels.
From the top of a cliff, where the ice melts, the sound of silence enveloping the vast landscape as the wind carried the frozen landscape into the


In [ ]:
test_prompt = "majestic waterfall"
x1 = generate_text(test_prompt, model3, tokenizer)
print('\n\n\n')
print(x1)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.






majestic waterfall in the heart of town, its cascading waterfalls filling the quaint night sky with magical light.


## PANO GENERATION PIPELINE

### SLOW DOWNLOAD IMPACT

In [ ]:
%cd /content
!git clone https://github.com/MukulRay1603/VRPANOGEN
%cd /content/VRPANOGEN

/content
Cloning into 'VRPANOGEN'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 10), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (35/35), 59.18 MiB | 23.00 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/VRPANOGEN


In [ ]:
# Main branch
!cp -r "/content/drive/MyDrive/PANOVR/Models/SDT2I" "/content/VRPANOGEN/SDT2I"

In [ ]:
!pip install -U diffusers==0.18.0 transformers accelerate xformers==0.0.16 realesrgan==0.3.0 torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 torchtext==0.14.1
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!pip install /content/VRPANOGEN
!pip install -q gradio gradio-pannellum

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.13.1 requires torch==1.13.1, but you have torch 2.1.0 which is incompatible.
torchdata 0.5.1 requires torch==1.13.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.14.1 requires torch==1.13.1, but you have torch 2.1.0 which is incompatible.
txt2panoimg 1.0.0 requires torch==1.13.1, but you have torch 2.1.0 which is incompatible.
txt2panoimg 1.0.0 requires xformers==0.0.16, but you have xformers 0.0.22.post7 which is incompatible.
Processing /content/VRPANOGEN
  Preparing metadata (setup.py) ... done
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Using cached xformers-0.0.16-cp310-cp310-manylinux2014_x86_64.whl (50.9 MB)
  Created wheel for txt2panoimg: filename=txt2panoim

In [ ]:
%cd /content/VRPANOGEN
import torch
from txt2panoimg import Text2360PanoramaImagePipeline
txt2panoimg = Text2360PanoramaImagePipeline('./SDT2I', torch_dtype=torch.float16, enable_xformers_memory_efficient_attention=True)

/content/VRPANOGEN


/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


### SLIGHTLY MODIFIED FASTER

In [ ]:
#DUP 2
%cd /content/drive/MyDrive/PANOVR/Code
!git clone https://github.com/MukulRay1603/VRPANOGEN
%cd /content/drive/MyDrive/PANOVR/Code/VRPANOGEN

/content/drive/MyDrive/PANOVR/Code
fatal: destination path 'VRPANOGEN' already exists and is not an empty directory.
/content/drive/MyDrive/PANOVR/Code/VRPANOGEN


In [ ]:
#DUP 2
!pip install -U diffusers==0.18.0 transformers accelerate xformers==0.0.16 realesrgan==0.3.0 torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 torchtext==0.14.1
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!pip install /content/drive/MyDrive/PANOVR/Code/VRPANOGEN
!pip install -q gradio gradio-pannellum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 112.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 26.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.9 MB/s eta 0

In [ ]:
#DUP 2
%cd /content/drive/MyDrive/PANOVR
import torch
from txt2panoimg import Text2360PanoramaImagePipeline
txt2panoimg = Text2360PanoramaImagePipeline('/content/drive/MyDrive/PANOVR/Models/SDT2I', torch_dtype=torch.float16, enable_xformers_memory_efficient_attention=True)

/content/drive/MyDrive/PANOVR


/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


# MAIN RUN INTERFACE

### Record and Transcribe

In [ ]:
button = widgets.Button(description="Start recording...")
button.on_click(_record_audio)
display(button)

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


In [ ]:
text = whisper_model.transcribe("/content/VRPANOGEN/recorded_audio.wav")
text['text']
voice_prompt = text['text']
voice_prompt

' Beautiful waterfall'

### Prompt generation with GPT NEO

In [ ]:
# Test the function with a custom prompt
# test_prompt = "The waterfall landscape"
# pano_input_prompt = generate_text(test_prompt, model3, tokenizer)
# print(pano_input_prompt)

pano_input_prompt = generate_text(voice_prompt, model3, tokenizer)
print(pano_input_prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Beautiful waterfall engulfs the landscape, its waters reflecting the sun’s rays throughout the year.


### Panorama generation with SD Pipeline

In [ ]:
# pano_input_prompt = "The waterfall landscape"

In [ ]:
import gradio as gr
from gradio_pannellum import Pannellum
from google.colab import files

In [ ]:
def generate(pano_input_prompt):
  input = {'prompt': pano_input_prompt, 'upscale': False}
  output = txt2panoimg(input)
  output.save('result.png')
  # files.download('result.png')
  return output

with gr.Blocks(css="style.css", title="PANORAMA VR 360 MODEL") as demo:
    with gr.Column():
        textbox = gr.Textbox(show_label=False, value=pano_input_prompt)
        button = gr.Button(value="Generate")

    with gr.Row(variant="default"):
        output_image = Pannellum(show_label=False, interactive=True, show_download_button=True)
        button.click(fn=generate, inputs=[textbox], outputs=[output_image], show_progress = True)


# Launch the demo
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://524208ad8ab01cac13.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
files.download('result.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>